#### This notebook contains code used to populate data dictionary
It is in development state and not cleaned at all!

In [1]:
import pandas as pd
from pandas_datareader import wb
from data_dictionary.append_df_to_excel import append_df_to_excel
# from openpyxl import load_workbook

C:\Users\beto\anaconda3\lib\site-packages\pandas_datareader\compat\__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [5]:
# read the excel tmee database
path_file = './populate_data_dictionary/TM Indicators Flow Revised Database - 2020306 SP ED ECD -working file v-6 Aug 2020_beto.xlsx'
indicators = pd.read_excel(path_file, sheet_name='TM Revised database', header = 2)

In [ ]:
# filter indicators that are new or retained from excel tmee data base
logic_not_removed = indicators['Status in the new `db'].str.lower().str.contains('new|retained')
# filter logic_not_removed that are not boolean
logic_not_null = logic_not_removed.notnull()
# and operator from logic above
logic_not_removed_null = logic_not_removed & logic_not_null
not_removed_indicators = indicators[logic_not_removed_null]

In [ ]:
# point to indicators with helix: or helix code: in excel tmee data base
logic_helix = not_removed_indicators['Data Source'].str.lower().str.contains('helix:|helix code:')
# filter logic_helix that are not boolean
logic_not_null = logic_helix.notnull()
# and operator from logic above
logic_helix_not_null = logic_helix & logic_not_null
not_removed_indicators['Code'] = not_removed_indicators[logic_helix_not_null]['Data Source']

In [ ]:
# keep only the code for API requests
not_removed_indicators['Code'] = not_removed_indicators['Code'].str.replace('Helix: ','')\
.str.replace('Helix code: ','').str.replace('/', '').str.replace('\\', '')

In [ ]:
# use a function from stackoverflow to append data dictionary
path_file = './populate_data_dictionary/indicator_dictionary_TM_v1.xlsx'
append_df_to_excel(path_file, not_removed_indicators.iloc[21:,[2,105]], sheet_name='Indicator', startrow=22,
                   startcol = 4, header = False, index = False)

In [ ]:
# standardize helix Api names as Helix for source names
not_removed_indicators['Source Name'] = 'Helix: ' + not_removed_indicators[logic_helix_not_null]['Code']

In [ ]:
# place list of helix api's in source
path_file = './populate_data_dictionary/indicator_dictionary_TM_v1.xlsx'
append_df_to_excel(path_file, not_removed_indicators.loc[logic_helix_not_null,'Source Name'].iloc[2:],
                   sheet_name='Source', startrow=6,
                   startcol = 2, header = False, index = False)

In [ ]:
not_removed_indicators.loc[logic_helix_not_null,'Source Name'].iloc[2:]

In [ ]:
not_removed_indicators.loc[logic_helix_not_null,'Code'].iloc[2:]

In [ ]:
not_removed_indicators.iloc[21:,-1]

In [ ]:
# keep only the code for API requests
not_removed_indicators['Code'].str.replace('Helix: ','').str.replace('Helix code: ','').str.replace('/', '')

In [ ]:
# use a function from stackoverflow
path_file = './populate_data_dictionary/indicator_dictionary_TM_v1.xlsx'
append_df_to_excel(path_file, not_removed_indicators.iloc[21:,-1], sheet_name='Indicator', startrow=22,
                   startcol = 5, header = False, index = False)

In [ ]:
len(helix_source_indicators)

In [ ]:
# now I want to plug correctly the codes in the populated Excel data dictionary
# should I join these two tables on name?
not_removed_indicators

In [ ]:
# write the indicator names of these into Excel (ideally to populate our dictionary)
# this solution does not properly work (creates a new spreadsheet)

# path to existing excel file
# I know before hand which indicators I want to add and the startrow and starcol parameters
path_file = './populate_data_dictionary/indicator_dictionary_TM_v1.xlsx'
with pd.ExcelWriter(path_file, mode='a', engine="openpyxl") as writer:
    indicators[logic_not_removed_null].iloc[21:,2].to_excel(writer, sheet_name='Indicator',
                                                                       header = False, index = False,
                                                                       startrow = 22, startcol = 4)

In [ ]:
# write the codes from helix API based indicators


In [ ]:
# get info from new or retained indicators
indicators

In [ ]:
# function from stackoverflow by MaxU
def append_df_to_excel(filename, df, sheet_name='Sheet1', startrow=None,
                       truncate_sheet=False, 
                       **to_excel_kwargs):
    """
    Append a DataFrame [df] to existing Excel file [filename]
    into [sheet_name] Sheet.
    If [filename] doesn't exist, then this function will create it.

    Parameters:
      filename : File path or existing ExcelWriter
                 (Example: '/path/to/file.xlsx')
      df : dataframe to save to workbook
      sheet_name : Name of sheet which will contain DataFrame.
                   (default: 'Sheet1')
      startrow : upper left cell row to dump data frame.
                 Per default (startrow=None) calculate the last row
                 in the existing DF and write to the next row...
      truncate_sheet : truncate (remove and recreate) [sheet_name]
                       before writing DataFrame to Excel file
      to_excel_kwargs : arguments which will be passed to `DataFrame.to_excel()`
                        [can be dictionary]

    Returns: None
    """
    from openpyxl import load_workbook

    import pandas as pd

    # ignore [engine] parameter if it was passed
    if 'engine' in to_excel_kwargs:
        to_excel_kwargs.pop('engine')

    writer = pd.ExcelWriter(filename, engine='openpyxl')

    # Python 2.x: define [FileNotFoundError] exception if it doesn't exist 
    try:
        FileNotFoundError
    except NameError:
        FileNotFoundError = IOError


    try:
        # try to open an existing workbook
        writer.book = load_workbook(filename)

        # get the last row in the existing Excel sheet
        # if it was not specified explicitly
        if startrow is None and sheet_name in writer.book.sheetnames:
            startrow = writer.book[sheet_name].max_row

        # truncate sheet
        if truncate_sheet and sheet_name in writer.book.sheetnames:
            # index of [sheet_name] sheet
            idx = writer.book.sheetnames.index(sheet_name)
            # remove [sheet_name]
            writer.book.remove(writer.book.worksheets[idx])
            # create an empty sheet [sheet_name] using old index
            writer.book.create_sheet(sheet_name, idx)

        # copy existing sheets
        writer.sheets = {ws.title:ws for ws in writer.book.worksheets}
    except FileNotFoundError:
        # file does not exist yet, we will create it
        pass

    if startrow is None:
        startrow = 0

    # write out the new sheet
    df.to_excel(writer, sheet_name, startrow=startrow, **to_excel_kwargs)

    # save the workbook
    writer.save()

#### dev section

In [ ]:
# test append df to excel function
path_file = './populate_data_dictionary/test.xlsx'
append_df_to_excel(path_file, not_removed_indicators.iloc[21:,[2,105]], sheet_name='Indicator', startrow=22,
                   startcol = 4, header = False, index = False)

In [ ]:
# looking forward to get more sources that could be extracted as API
not_removed_indicators['Data Source'].unique()

In [ ]:
wb.search('health.*expenditure.*gdp')

In [ ]:
wb.download(indicator='SH.XPD.CHEX.GD.ZS', country=['AL', 'AM', 'TR', 'TM', 'BA'], start=1960, end=2020)

In [ ]:
# filter indicators that are new or retained from excel tmee data base
logic_not_removed = indicators['Status in the new `db'].str.lower().str.contains('new|retained')
# filter logic_not_removed that are not boolean
logic_not_null = logic_not_removed.notnull()

In [ ]:
len(indicators[logic_not_removed & logic_not_null])

In [ ]:
# open existing excel file
path_file = './populate_data_dictionary/indicator_dictionary_TM_v1.xlsx'
# book = load_workbook(path_file)

In [ ]:
# open excel writer
writer = pd.ExcelWriter(path_file, engine='openpyxl')


In [ ]:
# open existing excel file
path_file = './populate_data_dictionary/indicator_dictionary_TM_v1.xlsx'
with pd.ExcelWriter(path_file, mode='a', engine="openpyxl") as writer:
    indicators[logic_not_removed & logic_not_null].iloc[21:,2].to_excel(writer, sheet_name='Indicator',
                                                                       header = False, index = False,
                                                                       startrow = 22, startcol = 4)
# df.to_excel(writer, sheet_name='Sheet3')

In [ ]:
# point to indicators with data source not null
logic_not_null = indicators.iloc[:,5].notnull()
# get those that contains 'helix' in data source
logic_helix = indicators[logic_not_null]['Data Source'].str.lower().str.contains('helix')

In [ ]:
# point to indicators with helix: or helix code: in excel tmee data base
logic_helix = indicators['Data Source'].str.lower().str.contains('helix:|helix code:')
# filter logic_helix that are not boolean
logic_not_null = logic_helix.notnull()

In [ ]:
indicators[logic_helix & logic_not_null]

In [ ]:
indicators[logic_not_removed_null].iloc[21:,2]

In [ ]:
indicators['Data Source'].unique()

In [ ]:
indicators['Status in the new `db'].unique()

In [ ]:
logic_not_removed = indicators['Status in the new `db'].str.lower().str.contains('new|retained')
indicators[logic_not_removed.isnull()].head(50)

In [ ]:
pd.Series(indicators['Status in the new `db'].unique()).str.lower().str.contains('new|retained')

In [ ]:
indicators.tail()